In [5]:
import sys
import os

# Get the current directory of the notebook
# and go up one level to the project root ('KINETICCONSTANTS_TEST')
project_root = os.path.abspath("..")

# Add the project root to the system path if it's not there already
if project_root not in sys.path:
    sys.path.append(project_root)

In [7]:
from torch_geometric.data import Data, Dataset, Batch
from torch_geometric.loader import DataLoader
from torch_geometric.utils import to_dense_adj, dense_to_sparse
import torch
import torch.nn as nn
import lmdb
import pickle
from tqdm import tqdm
import numpy as np
import pandas as pd
from model import SimpleGNN
import os
from simulation.produce_simulations import SimulatedGraphDataset

In [10]:
ckpt_dir = '../model_checkpoints'  # Directory to load checkpoints
ckpt = 4
model = SimpleGNN(in_channels=1000, hidden_channels=128, node_out_channels=4, edge_out_channels=2)
model.load_state_dict(torch.load(os.path.join(ckpt_dir, f'model_checkpoint_epoch_{ckpt}.pt')))
model.eval()

SimpleGNN(
  (embed): mlp(
    (mlp): Sequential(
      (0): Linear(in_features=1000, out_features=256, bias=True)
      (1): PReLU(num_parameters=1)
      (2): Dropout(p=0.1, inplace=False)
      (3): Linear(in_features=256, out_features=256, bias=True)
      (4): PReLU(num_parameters=1)
      (5): Dropout(p=0.1, inplace=False)
      (6): Linear(in_features=256, out_features=256, bias=True)
      (7): PReLU(num_parameters=1)
      (8): Linear(in_features=256, out_features=128, bias=True)
    )
  )
  (graph_encoder): mlp(
    (mlp): Sequential(
      (0): Linear(in_features=128, out_features=256, bias=True)
      (1): PReLU(num_parameters=1)
      (2): Dropout(p=0.1, inplace=False)
      (3): Linear(in_features=256, out_features=256, bias=True)
      (4): PReLU(num_parameters=1)
      (5): Dropout(p=0.1, inplace=False)
      (6): Linear(in_features=256, out_features=256, bias=True)
      (7): PReLU(num_parameters=1)
      (8): Linear(in_features=256, out_features=128, bias=True)
    )


In [12]:
db_path = '../simulation/simulated_graph_dataset'
dataset = SimulatedGraphDataset(root=db_path)
torch.manual_seed(42)
dataset = dataset.shuffle()  # Shuffle the dataset
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = dataset[:train_size], dataset[train_size:]

batch_size = 32  # Adjust batch size as needed
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=0)

In [34]:
adj_matrix_df = pd.read_csv('../simulation/adjacency_matrix.csv', index_col=0)
all_lipids = adj_matrix_df.index.tolist()
adj_matrix = np.array(adj_matrix_df)
num_edges = np.sum(adj_matrix, dtype=int)
correlation_matrix_partial = pd.read_csv('../simulation/correlation_matrix.csv', index_col=0)
correlation_matrix = pd.DataFrame(np.eye(len(all_lipids)), index=all_lipids, columns=all_lipids)
L = np.linalg.cholesky(correlation_matrix.values)

ks_pred = {f'k_{idx}': [] for idx in range(num_edges)}
ks_true = {f'k_{idx}': [] for idx in range(num_edges)}

with torch.no_grad():
    for batch in tqdm(test_loader, desc="Evaluating"):
        data = batch.to('cpu')
        out_node, out_edge = model(data.x, data.edge_index , data.batch)
        k_prod, k_deg, sigma_conc, dropout = out_node.split([1, 1, 1, 1], dim=-1)
        k, sigma_k = out_edge.split([1, 1], dim=-1)
        row, col = data.edge_index
        row = row[:num_edges]
        col = col[:num_edges]
        print(np.array(data.parameters['kinetic_constants']).shape)
        print(np.array(data.parameters['kinetic_constants'])[:][32].shape)
        
        for idx in range(num_edges):
            ks_pred[f'k_{idx}'].append(list(k[idx::num_edges]))
            ks_true[f'k_{idx}'].append(np.array(data.parameters['kinetic_constants'])[:][row[idx]][col[idx]])

Evaluating:   0%|          | 0/63 [00:00<?, ?it/s]

(32, 113, 113)


IndexError: index 32 is out of bounds for axis 0 with size 32

In [ ]:
import matplotlib.pyplot as plt

fig, axs = plt.subplots(nrows=4, ncols=5, figsize=(20, 16))
for idx in range(20):
    row = idx // 5
    col = idx % 5
    axs[row, col].scatter(ks_true[f'k_{idx}'], ks_pred[f'k_{idx}'], alpha=0.5)
    axs[row, col].plot([min(ks_true[f'k_{idx}']), max(ks_true[f'k_{idx}'])],
                       [min(ks_true[f'k_{idx}']), max(ks_true[f'k_{idx}'])], 'r--')
    axs[row, col].set_title(f'Kinetic Constant k_{idx}')
    axs[row, col].set_xlabel('True Values')
    axs[row, col].set_ylabel('Predicted Values')